In [149]:
import $ivy.`org.tpolecat::doobie-core:0.8.8`
import $ivy.`org.postgresql:postgresql:42.2.10`
import $ivy.`org.tpolecat::doobie-postgres:0.8.8`

Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-postgres_2.13/0.8.8/doobie-postgres_2.13-0.8.8.pom
Downloaded https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.9/postgresql-42.2.9.pom
Downloaded https://repo1.maven.org/maven2/co/fs2/fs2-io_2.13/2.1.0/fs2-io_2.13-2.1.0.pom
Downloaded https://repo1.maven.org/maven2/co/fs2/fs2-io_2.13/2.1.0/fs2-io_2.13-2.1.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-postgres_2.13/0.8.8/doobie-postgres_2.13-0.8.8-sources.jar
Downloaded https://repo1.maven.org/maven2/co/fs2/fs2-io_2.13/2.1.0/fs2-io_2.13-2.1.0.jar
Downloaded https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.9/postgresql-42.2.9-sources.jar
Downloaded https://repo1.maven.org/maven2/org/postgresql/postgresql/42.2.9/postgresql-42.2.9.jar
Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-postgres_2.13/0.8.8/doobie-postgres_2.13-0.8.8.jar


import $ivy.$                                

import $ivy.$                                  

import $ivy.$                                    

In [77]:
import cats._, cats.data._, cats.effect._, cats.implicits._
import doobie._, doobie.implicits._
import fs2.Stream

import cats._, cats.data._, cats.effect._, cats.implicits._

import doobie._, doobie.implicits._

import fs2.Stream

In [43]:
import doobie.util.ExecutionContexts
implicit val cs = IO.contextShift(ExecutionContexts.synchronous)
val xa = Transactor.fromDriverManager[IO](
  "org.postgresql.Driver",     // driver classname
  "jdbc:postgresql:world",     // connect URL (driver-specific)
  "postgres",                  // user
  "",                          // password
  Blocker.liftExecutionContext(ExecutionContexts.synchronous) // just for testing
)

import doobie.util.ExecutionContexts

cs: ContextShift[IO] = cats.effect.internals.IOContextShift@5a19259b
xa: Transactor.Aux[IO, Unit] = doobie.util.transactor$Transactor$$anon$13@7d22e762

In [47]:
val program1 = 42.pure[ConnectionIO]
val io = program1.transact(xa)
io.unsafeRunSync

program1: ConnectionIO[Int] = Pure(42)
io: IO[Int] = Async(
  cats.effect.internals.IOBracket$$$Lambda$2863/259606418@48130d6e,
  false
)
res46_2: Int = 42

In [48]:
val program2 = sql"select 42".query[Int].unique
program2.transact(xa).unsafeRunAsync(println)

Right(42)


program2: ConnectionIO[Int] = Suspend(
  BracketCase(
    Suspend(PrepareStatement("select 42")),
    doobie.hi.connection$$$Lambda$3025/964657051@7d018e80,
    cats.effect.Bracket$$Lambda$3027/2114759256@467139e8
  )
)

In [49]:
val program3: ConnectionIO[(Int, Double)] =
  for {
    a <- sql"select 42".query[Int].unique
    b <- sql"select random()".query[Double].unique
  } yield (a, b)
program3.transact(xa).unsafeRunSync

program3: ConnectionIO[(Int, Double)] = FlatMapped(
  Suspend(
    BracketCase(
      Suspend(PrepareStatement("select 42")),
      doobie.hi.connection$$$Lambda$3025/964657051@320494d7,
      cats.effect.Bracket$$Lambda$3027/2114759256@e77a4d
    )
  ),
  ammonite.$sess.cmd48$Helper$$Lambda$3587/1129877496@6fb34d72
)
res48_1: (Int, Double) = (42, 0.9449415732984399)

In [50]:
val program3a = {
  val a: ConnectionIO[Int] = sql"select 42".query[Int].unique
  val b: ConnectionIO[Double] = sql"select random()".query[Double].unique
  (a, b).tupled
}
program3a.transact(xa).unsafeRunSync

program3a: ConnectionIO[(Int, Double)] = FlatMapped(
  FlatMapped(
    Suspend(
      BracketCase(
        Suspend(PrepareStatement("select 42")),
        doobie.hi.connection$$$Lambda$3025/964657051@45170a9a,
        cats.effect.Bracket$$Lambda$3027/2114759256@327230be
      )
    ),
    cats.FlatMap$$Lambda$3018/1974523915@5ffd5405
  ),
  cats.Monad$$Lambda$2798/2001364205@75255920
)
res49_1: (Int, Double) = (42, 0.9301555440877713)

In [36]:
val program3a = {
  val a: ConnectionIO[Int] = sql"select 42".query[Int].unique
  val b: ConnectionIO[Double] = sql"select random()".query[Double].unique
  (a, b).tupled
}
program3a.replicateA(5).transact(xa).unsafeRunSync

program3a: ConnectionIO[(Int, Double)] = FlatMapped(
  FlatMapped(
    Suspend(
      BracketCase(
        Suspend(PrepareStatement("select 42")),
        doobie.hi.connection$$$Lambda$3025/964657051@1b4aa139,
        cats.effect.Bracket$$Lambda$3027/2114759256@683a72d1
      )
    ),
    cats.FlatMap$$Lambda$3018/1974523915@3cdf46d7
  ),
  cats.Monad$$Lambda$2798/2001364205@4f187c5a
)
res35_1: List[(Int, Double)] = List(
  (42, 0.25058705885668786),
  (42, 0.9150082867816458),
  (42, 0.45798019903125464),
  (42, 0.461622284297345),
  (42, 0.5036114233007609)
)

# SELECTING DATA

In [65]:
sql"select name from country"
  .query[String]    // Query0[String]
  .to[List]         // ConnectionIO[List[String]]
  .transact(xa)     // IO[List[String]]
  .unsafeRunSync    // List[String]
  .take(5)          // List[String]
  .foreach(println) // Unit

Afghanistan
Netherlands
Netherlands Antilles
Albania
Algeria


In [64]:
sql"""select name from country where code='AFG'"""
  .query[String]    // Query0[String]
  .unique         // ConnectionIO[List[String]]
  .transact(xa)     // IO[List[String]]
  .unsafeRunSync    // List[String]


res63: String = "Afghanistan"

In [66]:
sql"select name from country"
  .query[String]    // Query0[String]
  .stream           // Stream[ConnectionIO, String]
  .take(5)          // Stream[ConnectionIO, String]
  .compile.toList   // ConnectionIO[List[String]]
  .transact(xa)     // IO[List[String]]
  .unsafeRunSync    // List[String]
  .foreach(println) // Unit

Afghanistan
Netherlands
Netherlands Antilles
Albania
Algeria


In [67]:
val y = xa.yolo // a stable reference is required
import y._

y: util.yolo.Yolo[IO] = doobie.util.yolo$Yolo@75a28c61
import y._

In [68]:
sql"select name from country"
  .query[String] // Query0[String]
  .stream        // Stream[ConnectionIO, String]
  .take(5)       // Stream[ConnectionIO, String]
  .quick         // IO[Unit]
  .unsafeRunSync

  Afghanistan
  Netherlands
  Netherlands Antilles
  Albania
  Algeria


In [71]:
sql"select code, name, population, gnp from country"
  .query[(String, String, Int, Option[Double])]
/*  .stream
  .take(5)
  .quick
  .unsafeRunSync
*/

res70: Query0[(String, String, Int, Option[Double])] = doobie.util.query$Query$$anon$3@602d6e83

In [72]:
import shapeless._

sql"select code, name, population, gnp from country"
  .query[String :: String :: Int :: Option[Double] :: HNil]
  .stream
  .take(5)
  .quick
  .unsafeRunSync

  AFG :: Afghanistan :: 22720000 :: Some(5976.0) :: HNil
  NLD :: Netherlands :: 15864000 :: Some(371362.0) :: HNil
  ANT :: Netherlands Antilles :: 217000 :: Some(1941.0) :: HNil
  ALB :: Albania :: 3401200 :: Some(3205.0) :: HNil
  DZA :: Algeria :: 31471000 :: Some(49982.0) :: HNil


import shapeless._



In [73]:
case class Country(code: String, name: String, pop: Int, gnp: Option[Double])

defined class Country

In [74]:
sql"select code, name, population, gnp from country"
  .query[Country]
  .stream
  .take(5)
  .quick
  .unsafeRunSync

  Country(AFG,Afghanistan,22720000,Some(5976.0))
  Country(NLD,Netherlands,15864000,Some(371362.0))
  Country(ANT,Netherlands Antilles,217000,Some(1941.0))
  Country(ALB,Albania,3401200,Some(3205.0))
  Country(DZA,Algeria,31471000,Some(49982.0))


In [75]:
case class Code(code: String)
case class Country2(name: String, pop: Int, gnp: Option[Double])

defined class Code
defined class Country2

In [76]:
sql"select code, name, population, gnp from country"
  .query[(Code, Country2)]
  .stream
  .take(5)
  .quick
  .unsafeRunSync

  (Code(AFG),Country2(Afghanistan,22720000,Some(5976.0)))
  (Code(NLD),Country2(Netherlands,15864000,Some(371362.0)))
  (Code(ANT),Country2(Netherlands Antilles,217000,Some(1941.0)))
  (Code(ALB),Country2(Albania,3401200,Some(3205.0)))
  (Code(DZA),Country2(Algeria,31471000,Some(49982.0)))


In [78]:
val p: Stream[IO, Country2] = {
  sql"select name, population, gnp from country"
    .query[Country2] // Query0[Country2]
    .stream          // Stream[ConnectionIO, Country2]
    .transact(xa)    // Stream[IO, Country2]
}
// p: Stream[IO, Country2] = Stream(..)

p.take(5).compile.toVector.unsafeRunSync.foreach(println)

Country2(Afghanistan,22720000,Some(5976.0))
Country2(Netherlands,15864000,Some(371362.0))
Country2(Netherlands Antilles,217000,Some(1941.0))
Country2(Albania,3401200,Some(3205.0))
Country2(Algeria,31471000,Some(49982.0))


p: Stream[IO, Country2] = Stream(..)

In [79]:
val proc = HC.stream[(Code, Country2)](
  "select code, name, population, gnp from country", // statement
  ().pure[PreparedStatementIO],                      // prep (none)
  512                                                // chunk size
)
// proc: Stream[ConnectionIO, (Code, Country2)] = Stream(..)

proc.take(5)        // Stream[ConnectionIO, (Code, Country2)]
    .compile.toList // ConnectionIO[List[(Code, Country2)]]
    .map(_.toMap)   // ConnectionIO[Map[Code, Country2]]
    .quick
    .unsafeRunSync

  HashMap(Code(ANT) -> Country2(Netherlands Antilles,217000,Some(1941.0)), Code(ALB) -> Country2(Albania,3401200,Some(3205.0)), Code(DZA) -> Country2(Algeria,31471000,Some(49982.0)), Code(NLD) -> Country2(Netherlands,15864000,Some(371362.0)), Code(AFG) -> Country2(Afghanistan,22720000,Some(5976.0)))


proc: Stream[ConnectionIO, (Code, Country2)] = Stream(..)

# PARAMETERIZED QUERIES

In [88]:
def biggerThan(minPop: Int) = sql"""
  select code, name, population, gnp
  from country
  where population > $minPop
""".query[Country]

defined function biggerThan

In [89]:
biggerThan(150000000).quick.unsafeRunSync

  Country(BRA,Brazil,170115000,Some(776739.0))
  Country(IDN,Indonesia,212107000,Some(84982.0))
  Country(IND,India,1013662000,Some(447114.0))
  Country(CHN,China,1277558000,Some(982268.0))
  Country(PAK,Pakistan,156483000,Some(61289.0))
  Country(USA,United States,278357000,Some(8510700.0))


In [90]:
def populationIn(range: Range) = sql"""
  select code, name, population, gnp
  from country
  where population > ${range.min}
  and   population < ${range.max}
""".query[Country]

populationIn(150000000 to 200000000).quick.unsafeRunSync

  Country(BRA,Brazil,170115000,Some(776739.0))
  Country(PAK,Pakistan,156483000,Some(61289.0))


defined function populationIn

In [91]:
def populationIn(range: Range, codes: NonEmptyList[String]) = {
  val q = fr"""
    select code, name, population, gnp
    from country
    where population > ${range.min}
    and   population < ${range.max}
    and   """ ++ Fragments.in(fr"code", codes) // code IN (...)
  q.query[Country]
}

defined function populationIn

In [93]:
populationIn(100000000 to 300000000, NonEmptyList.of("USA", "BRA", "PAK", "GBR")).quick.unsafeRunSync

  Country(BRA,Brazil,170115000,Some(776739.0))
  Country(PAK,Pakistan,156483000,Some(61289.0))
  Country(USA,United States,278357000,Some(8510700.0))


# TYPEChECKING QUERIES

In [94]:
def biggerThan(minPop: Short) =
  sql"""
    select code, name, population, gnp, indepyear
    from country
    where population > $minPop
  """.query[Country]

defined function biggerThan

In [95]:
biggerThan(0).check.unsafeRunSync

  Query0[$cmd93.cmd72.Country] defined at cmd93.sc:2
  select code, name, population, gnp, indepyear
  from country
  where population > ?
  ✓ SQL Compiles and TypeChecks
  ✕ P01 Short  →  INTEGER (int4)
    Short is not coercible to INTEGER (int4) according to the JDBC
    specification. Expected schema type was SMALLINT.
  ✓ C01 code       CHAR     (bpchar)  NOT NULL  →  String
  ✓ C02 name       VARCHAR  (varchar) NOT NULL  →  String
  ✓ C03 population INTEGER  (int4)    NOT NULL  →  Int
  ✕ C04 gnp        NUMERIC  (numeric) NULL      →  Option[Double]
    NUMERIC (numeric) is ostensibly coercible to Option[Double]
    according to the JDBC specification but is not a recommended
    target type. Expected schema type was FLOAT or DOUBLE.
  ✕ C05 indepyear  SMALLINT (int2)    NULL      →  
    Column is unused. Remove it from the SELECT statement.


In [98]:
sql"select name, population from country".query[String].check.unsafeRunSync

  Query0[String] defined at cmd97.sc:1
  select name, population from country
  ✓ SQL Compiles and TypeChecks
  ✓ C01 name       VARCHAR (varchar) NOT NULL  →  String
  ✕ C02 population INTEGER (int4)    NOT NULL  →  
    Column is unused. Remove it from the SELECT statement.


# DDL, Inserting, and Updating

In [99]:
val drop =
  sql"""
    DROP TABLE IF EXISTS person
  """.update.run

val create =
  sql"""
    CREATE TABLE person (
      id   SERIAL,
      name VARCHAR NOT NULL UNIQUE,
      age  SMALLINT
    )
  """.update.run

drop: ConnectionIO[Int] = Suspend(
  BracketCase(
    Suspend(
      PrepareStatement(
        """
    DROP TABLE IF EXISTS person
  """
      )
    ),
    doobie.hi.connection$$$Lambda$3025/964657051@262b243f,
    cats.effect.Bracket$$Lambda$3027/2114759256@26630c16
  )
)
create: ConnectionIO[Int] = Suspend(
  BracketCase(
    Suspend(
      PrepareStatement(
        """
    CREATE TABLE person (
      id   SERIAL,
      name VARCHAR NOT NULL UNIQUE,
      age  SMALLINT
    )
  """
      )
    ),
    doobie.hi.connection$$$Lambda$3025/964657051@68ae5d59,
    cats.effect.Bracket$$Lambda$3027/2114759256@3a6f3a3a
  )
)

In [100]:
(drop, create).mapN(_ + _).transact(xa).unsafeRunSync

res99: Int = 0

In [101]:
def insert1(name: String, age: Option[Short]): Update0 =
  sql"insert into person (name, age) values ($name, $age)".update

defined function insert1

In [102]:
insert1("Alice", Some(12)).run.transact(xa).unsafeRunSync
// res1: Int = 1
insert1("Bob", None).quick.unsafeRunSync // switch to YOLO mode
//   1 row(s) updated

  1 row(s) updated


res101_0: Int = 1

In [103]:
case class Person(id: Long, name: String, age: Option[Short])

defined class Person

In [104]:
def insert2(name: String, age: Option[Short]): ConnectionIO[Person] =
  for {
    _  <- sql"insert into person (name, age) values ($name, $age)".update.run
    id <- sql"select lastval()".query[Long].unique
    p  <- sql"select id, name, age from person where id = $id".query[Person].unique
  } yield p

defined function insert2

In [105]:
insert2("Jimmy", Some(42)).quick.unsafeRunSync


  Person(3,Jimmy,Some(42))


In [106]:
def insert3(name: String, age: Option[Short]): ConnectionIO[Person] = {
  sql"insert into person (name, age) values ($name, $age)"
    .update
    .withUniqueGeneratedKeys("id", "name", "age")
}

defined function insert3

In [107]:
insert3("Elvis", None).quick.unsafeRunSync

  Person(4,Elvis,None)


In [109]:
sql"select name from country as C where C.code = SPA".query[String].to[List]

res108: ConnectionIO[List[String]] = Suspend(
  BracketCase(
    Suspend(
      PrepareStatement("select name from country as C where C.code = SPA")
    ),
    doobie.hi.connection$$$Lambda$3025/964657051@167ca9f,
    cats.effect.Bracket$$Lambda$3027/2114759256@6a8d3d3e
  )
)

In [110]:
sql"select code from country".query[String].to[List]

res109: ConnectionIO[List[String]] = Suspend(
  BracketCase(
    Suspend(PrepareStatement("select code from country")),
    doobie.hi.connection$$$Lambda$3025/964657051@4be3a465,
    cats.effect.Bracket$$Lambda$3027/2114759256@2b7467c9
  )
)

In [120]:
implicit val han = LogHandler.jdkLogHandler

han: LogHandler = LogHandler(
  doobie.util.log$LogHandler$$$Lambda$2795/2070485161@689fa92e
)

In [125]:
val fmquery = for {
    codes <- sql"select code from country where population > 100000000".query[String].nel
    name <- (fr"select name from country as C where "++
        Fragments.in(fr"C.code", codes)).query[String].to[List]
} yield name

fmquery: cats.free.Free[free.connection.ConnectionOp, List[String]] = FlatMapped(
  Suspend(
    BracketCase(
      Suspend(
        PrepareStatement(
          "select code from country where population > 100000000"
        )
      ),
      doobie.hi.connection$$$Lambda$3025/964657051@2ad77508,
      cats.effect.Bracket$$Lambda$3027/2114759256@514c0238
    )
  ),
  ammonite.$sess.cmd124$Helper$$Lambda$4475/1530349478@7b0b1d57
)

In [127]:
fmquery.quick.unsafeRunSync

May 03, 2020 1:55:45 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  select code from country where population > 100000000

 arguments = []
   elapsed = 3 ms exec + 2 ms processing (5 ms total)
          
May 03, 2020 1:55:45 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  select name from country as C where  C.code IN (?, ?, ?, ?, ?, ?, ?, ?, ?, ?) 

 arguments = [BGD, BRA, IDN, IND, JPN, CHN, NGA, PAK, RUS, USA]
   elapsed = 2 ms exec + 0 ms processing (2 ms total)
          


  List(Bangladesh, Brazil, Indonesia, India, Japan, China, Nigeria, Pakistan, Russian Federation, United States)


# Statement Fragments

In [128]:
val a = fr"select name from country"
// a: Fragment = Fragment("select name from country ")
val b = fr"where code = 'USA'"
// b: Fragment = Fragment("where code = 'USA' ")
val c = a ++ b // concatenation by ++
// c: Fragment = Fragment("select name from country where code = 'USA' ") // concatenation by ++
c.query[String].unique.quick.unsafeRunSync
//   United States

May 03, 2020 1:57:30 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  select name from country where code = 'USA' 

 arguments = []
   elapsed = 3 ms exec + 0 ms processing (3 ms total)
          


  United States


a: Fragment = Fragment("select name from country ")
b: Fragment = Fragment("where code = 'USA' ")
c: Fragment = Fragment("select name from country where code = 'USA' ")

In [141]:
def whereCode(s: String) = fr"where code = $s"
val fra = whereCode("FRA")
// fra: Fragment = Fragment("where code = ? ")
//(fr"select name from country" ++ fra).query[String].quick.unsafeRunSync
 (sql"select name from country where code = 'FRA'").query[String].unique.transact(xa).unsafeRunSync

May 03, 2020 2:06:37 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  select name from country where code = 'FRA'

 arguments = []
   elapsed = 2 ms exec + 0 ms processing (2 ms total)
          


defined function whereCode
fra: Fragment = Fragment("where code = ? ")
res140_2: String = "France"

In [142]:
def count(table: String) = 
    (fr"select count(*) from" ++ 
      Fragment.const(table)
    ).query[Int].unique
count("city").quick.unsafeRunSync

May 03, 2020 2:07:12 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  select count(*) from city 

 arguments = []
   elapsed = 3 ms exec + 0 ms processing (4 ms total)
          


  4079


defined function count

In [143]:
fr"IN (" ++ List(1, 2, 3).map(n => fr"$n").intercalate(fr",") ++ fr")"
// res3: Fragment = Fragment("IN ( ? , ? , ? ) ")
fr0"IN (" ++ List(1, 2, 3).map(n => fr0"$n").intercalate(fr",") ++ fr")"
// res4: Fragment = Fragment("IN (?, ?, ?) ")

res142_0: Fragment = Fragment("IN ( ? , ? , ? ) ")
res142_1: Fragment = Fragment("IN (?, ?, ?) ")

# Error handling

In [144]:
List(
  sql"""DROP TABLE IF EXISTS person""",
  sql"""CREATE TABLE person (
          id    SERIAL,
          name  VARCHAR NOT NULL UNIQUE
        )"""
).traverse(_.update.quick).void.unsafeRunSync

May 03, 2020 2:18:41 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  DROP TABLE IF EXISTS person

 arguments = []
   elapsed = 12 ms exec + 0 ms processing (12 ms total)
          


  0 row(s) updated


May 03, 2020 2:18:42 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  CREATE TABLE person (
            id    SERIAL,
            name  VARCHAR NOT NULL UNIQUE
          )

 arguments = []
   elapsed = 11 ms exec + 0 ms processing (11 ms total)
          


  0 row(s) updated


In [145]:
case class Person(id: Int, name: String)

def insert(s: String): ConnectionIO[Person] = {
  sql"insert into person (name) values ($s)"
    .update
    .withUniqueGeneratedKeys("id", "name")
}

defined class Person
defined function insert

In [146]:
insert("bob").quick.unsafeRunSync

  Person(1,bob)


In [148]:
try {
  insert("bob").quick.unsafeRunSync
} catch {
  case e: java.sql.SQLException =>
    println(e.getMessage)
    println(e.getSQLState)
}

ERROR: duplicate key value violates unique constraint "person_name_key"
  Detail: Key (name)=(bob) already exists.
23505


In [150]:
import doobie.postgres._

def safeInsert(s: String): ConnectionIO[Either[String, Person]] =
  insert(s).attemptSomeSqlState {
    case sqlstate.class23.UNIQUE_VIOLATION => "Oops!"
  }

import doobie.postgres._


defined function safeInsert

In [151]:
safeInsert("bob").quick.unsafeRunSync

  Left(Oops!)


In [152]:
safeInsert("juan").quick.unsafeRunSync

  Right(Person(5,juan))


# SQL Arrays

In [155]:
import doobie.postgres._
import doobie.postgres.implicits._

import doobie.postgres._

import doobie.postgres.implicits._

In [156]:
val drop = sql"DROP TABLE IF EXISTS person".update.quick

val create =
  sql"""
    CREATE TABLE person (
      id   SERIAL,
      name VARCHAR   NOT NULL UNIQUE,
      pets VARCHAR[] NOT NULL
    )
  """.update.quick

drop: IO[Unit] = Async(
  cats.effect.internals.IOBracket$$$Lambda$2863/259606418@37459ed1,
  false
)
create: IO[Unit] = Async(
  cats.effect.internals.IOBracket$$$Lambda$2863/259606418@7b743345,
  false
)

In [157]:
(drop *> create).unsafeRunSync

May 03, 2020 4:44:17 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  DROP TABLE IF EXISTS person

 arguments = []
   elapsed = 3 ms exec + 0 ms processing (3 ms total)
          


  0 row(s) updated


May 03, 2020 4:44:17 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

      CREATE TABLE person (
        id   SERIAL,
        name VARCHAR   NOT NULL UNIQUE,
        pets VARCHAR[] NOT NULL
      )
    

 arguments = []
   elapsed = 19 ms exec + 0 ms processing (19 ms total)
          


  0 row(s) updated


In [158]:
case class Person(id: Long, name: String, pets: List[String])

def insert(name: String, pets: List[String]): ConnectionIO[Person] = {
  sql"insert into person (name, pets) values ($name, $pets)"
    .update
    .withUniqueGeneratedKeys("id", "name", "pets")
}

defined class Person
defined function insert

In [159]:
insert("Bob", List("Nixon", "Slappy")).quick.unsafeRunSync
//   Person(1,Bob,List(Nixon, Slappy))
insert("Alice", Nil).quick.unsafeRunSync

  Person(1,Bob,List(Nixon, Slappy))
  Person(2,Alice,List())


# Unit Testing

In [163]:
import $ivy.`org.scalatest::scalatest:3.1.1`
import $ivy.`org.tpolecat::doobie-scalatest:0.8.8`

Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-scalatest_2.13/0.8.8/doobie-scalatest_2.13-0.8.8.pom
Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-scalatest_2.13/0.8.8/doobie-scalatest_2.13-0.8.8.jar
Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-scalatest_2.13/0.8.8/doobie-scalatest_2.13-0.8.8-sources.jar


import $ivy.$                               

import $ivy.$                                     

In [164]:
case class Country(code: Int, name: String, pop: Int, gnp: Double)

val trivial =
  sql"""
    select 42, 'foo'::varchar
  """.query[(Int, String)]

def biggerThan(minPop: Short) =
  sql"""
    select code, name, population, gnp, indepyear
    from country
    where population > $minPop
  """.query[Country]

def update(oldName: String, newName: String) =
  sql"""
    update country set name = $newName where name = $oldName
  """.update

defined class Country
trivial: Query0[(Int, String)] = doobie.util.query$Query$$anon$3@328ab0ef
defined function biggerThan
defined function update

In [165]:
import org.scalatest._

class AnalysisTestScalaCheck extends FunSuite with Matchers with doobie.scalatest.IOChecker {

  override val colors = doobie.util.Colors.None // just for docs

  val transactor = Transactor.fromDriverManager[IO](
    "org.postgresql.Driver", "jdbc:postgresql:world", "postgres", ""
  )

  test("trivial")    { check(trivial)        }
  test("biggerThan") { check(biggerThan(0))  }
  test("update")     { check(update("", "")) }

}

import org.scalatest._


defined class AnalysisTestScalaCheck

In [167]:
(new AnalysisTestScalaCheck).execute()

cmd164$Helper$AnalysisTestScalaCheck:
- trivial
- biggerThan *** FAILED ***
  Query0[cmd164.cmd163.Country] defined at cmd163.sc:9
    select code, name, population, gnp, indepyear
    from country
    where population > ?
    ✓ SQL Compiles and TypeChecks
    ✕ P01 Short  →  INTEGER (int4)
      Short is not coercible to INTEGER (int4) according to the JDBC
      specification. Expected schema type was SMALLINT.
    ✕ C01 code       CHAR     (bpchar)  NOT NULL  →  Int
      CHAR (bpchar) is ostensibly coercible to Int according to the JDBC
      specification but is not a recommended target type. Expected
      schema type was INTEGER.
    ✓ C02 name       VARCHAR  (varchar) NOT NULL  →  String
    ✓ C03 population INTEGER  (int4)    NOT NULL  →  Int
    ✕ C04 gnp        NUMERIC  (numeric) NULL      →  Double
      NUMERIC (numeric) is ostensibly coercible to Double according to
      the JDBC specification but is not a recommended target type.
      Expected schema type was FLOAT or 

# Managing Connections

In [168]:
import $ivy.`org.tpolecat::doobie-hikari:0.8.8`

Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-hikari_2.13/0.8.8/doobie-hikari_2.13-0.8.8.pom
Downloaded https://repo1.maven.org/maven2/com/zaxxer/HikariCP/3.4.1/HikariCP-3.4.1.pom
Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-hikari_2.13/0.8.8/doobie-hikari_2.13-0.8.8-sources.jar
Downloaded https://repo1.maven.org/maven2/org/tpolecat/doobie-hikari_2.13/0.8.8/doobie-hikari_2.13-0.8.8.jar
Downloaded https://repo1.maven.org/maven2/com/zaxxer/HikariCP/3.4.1/HikariCP-3.4.1-sources.jar
Downloaded https://repo1.maven.org/maven2/com/zaxxer/HikariCP/3.4.1/HikariCP-3.4.1.jar


import $ivy.$                                  

In [169]:
import doobie.hikari._

import doobie.hikari._

In [173]:
val transactor: Resource[IO, HikariTransactor[IO]] =
    for {
      ce <- ExecutionContexts.fixedThreadPool[IO](32) // our connect EC
      be <- Blocker[IO]    // our blocking EC
      xa <- HikariTransactor.newHikariTransactor[IO](
          "org.postgresql.Driver",     // driver classname
          "jdbc:postgresql:world",     // connect URL (driver-specific)
          "postgres",                  // user
          "",                          // password
          ce,                          // await connection here
          be                           // execute JDBC operations here
            )
    } yield xa

transactor: Resource[IO, HikariTransactor[IO]] = Bind(
  Bind(
    Allocate(
      Map(
        Delay(doobie.util.ExecutionContexts$$$Lambda$4834/1654447853@7b7aa543),
        scala.Function1$$Lambda$332/1620823990@447296e9,
        1
      )
    ),
    cats.effect.Resource$$Lambda$2919/2101121789@5e6e8cbb
  ),
  ammonite.$sess.cmd172$Helper$$Lambda$4861/981580907@6b26885d
)

In [176]:
transactor.use { xa =>

      // Construct and run your server here!
      for {
        n <- sql"select 42".query[Int].unique.transact(xa)
        _ <- IO(println(n))
      } yield ExitCode.Success

    }.unsafeRunSync

May 03, 2020 5:12:33 PM doobie.util.log$LogHandler$ $anonfun$jdkLogHandler$1
INFO: Successful Statement Execution:

  select 42

 arguments = []
   elapsed = 1 ms exec + 0 ms processing (1 ms total)
          


42


res175: ExitCode = ExitCode(0)

# Quill Integration

In [178]:
import $ivy.`org.tpolecat::doobie-quill:0.8.8`
import $ivy.`io.getquill::quill-jdbc:3.5.1`

Downloaded https://repo1.maven.org/maven2/io/getquill/quill-jdbc_2.13/3.5.1/quill-jdbc_2.13-3.5.1.pom
Downloaded https://repo1.maven.org/maven2/io/getquill/quill-sql_2.13/3.5.1/quill-sql_2.13-3.5.1.pom
Downloaded https://repo1.maven.org/maven2/com/zaxxer/HikariCP/3.4.2/HikariCP-3.4.2.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.13/2.1.3/scala-collection-compat_2.13-2.1.3.pom
Downloaded https://repo1.maven.org/maven2/com/github/vertical-blank/scala-sql-formatter_2.13/1.0.1/scala-sql-formatter_2.13-1.0.1.pom
Downloaded https://repo1.maven.org/maven2/io/getquill/quill-core_2.13/3.5.1/quill-core_2.13-3.5.1.pom
Downloaded https://repo1.maven.org/maven2/com/github/vertical-blank/sql-formatter/1.0/sql-formatter-1.0.pom
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.13/2.1.3/scala-collection-compat_2.13-2.1.3.jar
Downloaded https://repo1.maven.org/maven2/com/github/vertical-blank/sql-formatter/1.0/sql

import $ivy.$                                 

import $ivy.$                              

In [182]:
import _root_.io.getquill.{ idiom => _, _ }
import doobie.quill.DoobieContext

import _root_.io.getquill.{ idiom => _, _ }

import doobie.quill.DoobieContext

In [183]:
val dc = new DoobieContext.Postgres(Literal) // Literal naming scheme
import dc._

dc: DoobieContext.Postgres[Literal.type] = doobie.quill.DoobieContext$Postgres@260b5f99
import dc._

In [184]:
case class Country(code: String, name: String, population: Int)

defined class Country

In [184]:
val q1 = quote { query[Country].filter(_.code == "GBR") }

cmd184.sc:1: exception during macro expansion: 
scala.reflect.macros.TypecheckException: value getquill is not a member of cats.effect.IO[Int]
	at scala.reflect.macros.contexts.Typers.$anonfun$typecheck$3(Typers.scala:44)
	at scala.reflect.macros.contexts.Typers.$anonfun$typecheck$2(Typers.scala:38)
	at scala.reflect.macros.contexts.Typers.doTypecheck$1(Typers.scala:37)
	at scala.reflect.macros.contexts.Typers.typecheck(Typers.scala:51)
	at scala.reflect.macros.contexts.Typers.typecheck$(Typers.scala:32)
	at scala.reflect.macros.contexts.Context.typecheck(Context.scala:18)
	at scala.reflect.macros.contexts.Context.typecheck(Context.scala:18)
	at io.getquill.dsl.QueryDslMacro.meta(QueryDslMacro.scala:23)
	at io.getquill.dsl.QueryDslMacro.expandEntity(QueryDslMacro.scala:11)

val q1 = quote { query[Country].filter(_.code == "GBR") }
                      ^Compilation Failed

: 

In [184]:
run(q1)

cmd184.sc:1: not found: value q1
val res184 = run(q1)
                 ^Compilation Failed

: 